In [1]:
import tarfile
import timeit
import random
import math
import re
import itertools
import datetime 
import os
import pickle
from operator import itemgetter
from scipy.signal import cheby2, resample, sosfilt
from scipy import signal
from scipy.io import loadmat, savemat
from tensorly import tensor as tensor_tly
from tensorly import norm, dot
from tensorly.decomposition import tucker
from tensorly.tucker_tensor import tucker_to_tensoravailable_databases
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras.activations import relu
from tensorflow.keras.models import load_model, Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten, BatchNormalization, Activation
from tensorflow.keras.regularizers import l2, l1_l2

import numpy as np
import pandas as pd
import ecg_plot
import seaborn as sns
import matplotlib.pyplot as plt
import logging
%matplotlib inline

2022-06-03 13:50:44.151915: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-03 13:50:44.151948: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/home/renan/Área de Trabalho/research-project/py37/lib/python3.7/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
class Database(object):
    def __init__(self, path, leads):
        self.path = path
        self.leads = leads
        self.freq = 0
        self.headers_path = []
        self.recordings_path = []


    def extract_from_drive(self):
        if not os.path.exists(self.path):
            with tarfile.open(self.path) as zip_file:
                zip_file.extractall()
    

    def load_hea_file(self, i):
        with open(self.headers_path[i], 'r') as f:
            hea_file = f.read()
        return hea_file


    def get_frequency(self):



    def load_paths(self):
        for f in sorted(os.listdir(self.path)):
            root, extension = os.path.splitext(f)
            if not root.startswith('.') and extension=='.hea':
                header_db_file = os.path.join(self.path, root + '.hea')
                recording_db_file = os.path.join(self.path, root + '.mat')
                if os.path.isfile(header_db_file) and os.path.isfile(recording_db_file):
                    self.headers_path.append(header_db_file)
                    self.recordings_path.append(recording_db_file)
        print(f'Found {len(self.headers_path)} recordings in {self.path}.')
    

class Diagnostic(object):
    diagnostics = []


    @classmethod
    def append_diagnostic(cls, diagnostic):
        cls.diagnostics.append(diagnostic)


    @classmethod
    def get_diagnostics(cls):
        return cls.diagnostics


    def __init__(self, diag_name, abbrev, code):
        self.name = diag_name
        self.abbrev = abbrev
        self.code = code
    
    

class DiagnosticDatabase(object):
    databases = []

    @classmethod
    def append_database(cls, diag_db):
        cls.databases.append(diag_db)     


    @classmethod
    def get_df_recordings(cls):    
        total_recs = []
        for diag_db in cls.databases:
                recs = [rec.__dict__ for rec in diag_db.recordings_diag]
                recs = [dict(rec, db=diag_db.db.path.split('/')[2], diagnostic=diag_db.diagnostic.abbrev) for rec in recs]
                total_recs.append(recs)
        total_recs = [rec for db in total_recs for rec in db]
        dataframe = pd.DataFrame(total_recs)
        return dataframe


    def __init__(self, diag_origin, db_origin):
        self.diagnostic = diag_origin
        self.db = db_origin
        self.headers_diag_path = []
        self.recordings_diag = []


    def get_labels(self, header):
        labels = list()
        for l in header.split('\n'):
            if l.startswith('#Dx'):
                try:
                    entries = l.split(': ')[1].split(',')
                    for entry in entries:
                        labels.append(entry.strip())
                except:
                    pass
        return labels


    def get_leads(self,header):
        leads = list()
        for i, l in enumerate(header.split('\n')):
            entries = l.split(' ')
            if i==0:
                num_leads = int(entries[1])
            elif i<=num_leads:
                leads.append(entries[-1])
            else:
                break
        return tuple(leads)


    def choose_leads(self, recording, header, leads):
        num_leads = len(leads)
        num_samples = np.shape(recording)[1]
        chosen_recording = np.zeros((num_leads, num_samples), recording.dtype)
        available_leads = self.get_leads(header)
        for i, lead in enumerate(leads):
            if lead in available_leads:
                j = available_leads.index(lead)
                chosen_recording[i, :] = recording[j, :]
        return chosen_recording


    def plot_ecg(self, index):
        ecg_plot.plot(self.rsp_cut_recordings_diag[index]/1000, sample_rate=self.db.freq/2, title='')
        ecg_plot.show()


class Record():
    def __init__(self, filename, inf, sup, data):
        self.filename = filename
        self.inf = inf
        self.sup = sup
        self.data = data

In [12]:
DiagnosticDatabase.databases = []
Diagnostic.diagnostics = []

af_diag = Diagnostic('Atrial Fibrilation', 'AF', '164889003')
sr_diag = Diagnostic('Sinus Rhythm', 'SR', '426783006')
Diagnostic.append_diagnostic(af_diag)
Diagnostic.append_diagnostic(sr_diag)

path_folder = '../databases/'
available_databases = os.listdir(path_folder)
available_databases.remove('.gitkeep')
available_databases.remove('WFDB_PTB')
available_databases.remove('WFDB_StPetersburg')
print(available_databases)

available_databases = list(map(lambda db: path_folder + db, available_databases))
leads = ('I', 'II', 'III', 'aVR', 'aVL', 'aVF', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6')

# Databases
for path in available_databases:
    folder = re.search('databases/(.*)', path).group(1)
    db = Database(path, leads)
    db.load_paths()
    db.get_frequency()
        
    # Diagnostics
    for diag in Diagnostic.get_diagnostics():
        diag_db = DiagnosticDatabase(diag, db)
        
        # Recordings
        for i, header_path in enumerate(diag_db.db.headers_path):
            header = diag_db.db.load_hea_file(i)
            labels = diag_db.get_labels(header)
            if diag_db.diagnostic.code in labels:
                # Get record
                rec_file = header_path.replace('hea', 'mat')
                recording = loadmat(rec_file)['val']
                recording = diag_db.choose_leads(recording, header, diag_db.db.leads)
                recording = np.array(recording, dtype=np.float64)
                
                # Filtering
                sos = signal.cheby2(12, 20, [0.35, 70], 
                                    'bandpass', 
                                    fs= diag_db.db.freq, 
                                    output='sos')
                for lead in range(0, recording.shape[0]):
                        to_filt = np.array(recording[lead, :], dtype=np.float64)
                        filtered = signal.sosfilt(sos, to_filt)
                        recording[lead, :] = np.array(filtered, dtype=np.float64)

                # Resample record to 250Hz
                new_freq = 250
                time_rec = len(recording[0])/diag_db.db.freq
                n_samples = int(time_rec*new_freq)
                recording = resample(recording, n_samples, axis=1)

                # Cut record in 250 samples
                interval = 250
                size_rec = recording.shape[1]
                samples_rec = math.floor(size_rec/interval)
                for i in range(0, samples_rec):
                    inf = i*interval
                    sup = ((i+1)*interval)
                    recording_interval = np.array(list(map(lambda lead: lead[inf:sup], recording)))
                    rec_filename = rec_file.split('/')[-1]
                    record = Record(rec_filename, 
                                    inf, 
                                    sup, 
                                    recording_interval)
                    diag_db.recordings_diag.append(record)
                diag_db.headers_diag_path.append(header_path)
        print(f' - Found {len(diag_db.headers_diag_path)} recordings for {diag_db.diagnostic.abbrev}.')
        print(f' - Unattached {len(diag_db.recordings_diag)} intervals.')
        DiagnosticDatabase.append_database(diag_db)

['WFDB_Ga', 'WFDB_Ningbo', 'WFDB_CPSC2018_2', 'WFDB_CPSC2018', 'WFDB_PTBXL', 'WFDB_ChapmanShaoxing']
Found 10344 recordings in ../databases/WFDB_Ga.
Frequency: 500.0


KeyboardInterrupt: 

### Statistics Database

In [5]:
df = DiagnosticDatabase.get_df_recordings()
print(f"TOTAL: {df.shape[0]} rows")
df = df.sample(frac=1, ignore_index=True, random_state=32)
print(f"AF: {str(df[df.diagnostic == 'AF'].shape[0])} rows")
print(f"SR: {str(df[df.diagnostic == 'SR'].shape[0])} rows")
df.groupby(['diagnostic', 'db']).size()

TOTAL: 343213 rows
AF: 54191 rows
SR: 289022 rows


diagnostic  db                  
AF          WFDB_CPSC2018            18306
            WFDB_CPSC2018_2           2325
            WFDB_ChapmanShaoxing     12730
            WFDB_Ga                   5690
            WFDB_PTBXL               15140
SR          WFDB_CPSC2018            14116
            WFDB_CPSC2018_2             61
            WFDB_ChapmanShaoxing     13500
            WFDB_Ga                  17435
            WFDB_Ningbo              62990
            WFDB_PTBXL              180920
dtype: int64

In [6]:
df.head()

,filename,inf,sup,data,db,diagnostic
0,Q3000.mat,1000,1250,"[[9.760849635141424, 12.143493689638246, 14.75...",WFDB_CPSC2018_2,AF
1,JS38372.mat,1750,2000,"[[-8.622005688806848, -12.716669437907457, -7....",WFDB_Ningbo,SR
2,HR20937.mat,1000,1250,"[[11.846834097678906, -5.77366283997905, -33.6...",WFDB_PTBXL,SR
3,HR00086.mat,1750,2000,"[[12.649530402341707, 15.416020621653105, 17.1...",WFDB_PTBXL,SR
4,JS09370.mat,2000,2250,"[[-6.000065873350305, -3.6633855946070715, -18...",WFDB_ChapmanShaoxing,AF


### Divide dataframe in pickles

In [8]:
div = int(len(df)/13)

for i in range(0, 13):
    start = div * i 
    final = div * (i + 1)
    df.iloc[start:final].to_pickle(f'../workdata/mu/part_{i}.pkl')

### Restart kernel (Flush RAM)

In [ ]:
os._exit(00)